9장
TMDB 5000 영화 데이터 세트: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata

MovieLens 데이터 세트: https://grouplens.org/datasets/movielens/latest/

### **데이터 로딩 및 가공**

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

movies = pd.read_csv('/content/drive/MyDrive/Kaggle - 파이썬 머신러닝 완벽 가이드/kaggleData/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [2]:
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count','popularity', 'keywords', 'overview']]

In [3]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


In [4]:
pd.set_option('max_colwidth', 100)
movies_df[['genres', 'keywords']][:1]
# 딕셔너리 내부의 장르 명은 딕서녀러의 key인 'name'으로 추출 가능

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


> ast - literal_eval

- 하나의 셀에 리스트 형태로 들어있는 문자열이 있으면, type이 str로 인식하기 때문에 이 컬럼을 for문 등의 조건문을 사용할 수 없다.

- 이를 해결하기 위한 방법으로 ast.literal_eval 함수를 사용할 수 있다.

- literal_eval 기능을 사용하기 위해선 ast 모듈을 알고 있어야 한다. ast는 Abstract Syntax Trees의 약자로 문법을 구조화 시켜주는 모듈이다.

### **크롤링 / DB upload**
- 웹 크롤링을 해서 CSV파일로 저장하게 되면 CSV상에선 분명히 리스트나 딕셔너리로 보였는데 type을 찍어보면 문자열로 변해있는 경우가 종종 발생한다.

- 또한 리스트로 잘 크롤링해왔더라도 저장되는 테이블과 연관된 Django의 model table이 charfield인 경우 무조건 문자열로 저장되서 나중에 post나 get을 이용한 views를 작성할 때 문제가 되는 경우가 발생한다.

### **literal_eval**
- 이럴때 손쉽게 쓸 수 있는게 literal_eval이다.
- literal_eval은 문자열을 딕셔너리/리스트 형태로 바꿔줄 수 있다. 사용방법은 다음과 같다.

```python
a = "['호랑이', '사자', '기린']"
a = liter_eval(a)
a = ['호랑이', '사자', '기린']
```



[eval vs literal_eval](https://pydole.tistory.com/entry/Python-eval%EA%B3%BC-literaleval-%EC%B0%A8%EC%9D%B4)

In [5]:
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [6]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


> apply(lambda x : [ y['name'] for y in x])
- x는 각 행의 'genres' 열의 값
- x는 여러 장르 정보를 포함하는 리스트들의 data
- 각 행의 'genres' 열에서 장르 이름을 추출하여 리스트로 반환

### **장르 기반 콘텐츠 필터링**

1. 문자열로 변환된 genres 칼럼을 Count 기반으로 피쳐 벡터화 변환
2. genres 문자열을 피처 벡터화 행렬로 변환한 data set을 코사인 유사도를 통해 비교 (이를 위해 data set 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성)
3. 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화를 추천

In [7]:
# genres를 문자열로 변경 => 이를 Countvectorizer로 피처 벡터화한 행렬 값을 코사인 유사도로 비교
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환.
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


### **COS Similarity를 이용한 코사인 유사도**

In [8]:
# 기준 행과 비교 행의 코사인 유사도를 행렬 형태로 반환
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:5])

# movies_df의 행별 장르 유사도 값을 가지는 data

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]
 [0.12598816 0.16903085 0.3380617  ... 0.12598816 0.         0.        ]
 [0.75592895 0.3380617  0.50709255 ... 0.         0.         0.        ]]


In [9]:
# genre_sim 객체를 이용하여 movies_df의 개별 레코드에 대해 가장 장르 유사도가 높은 순으로 다른 레코드를 추출
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
# 행은 그대로 유지하면서 각 행의 열의 순서를 거꾸로 바꾸는 것을 의미
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


- [::-1,:]은 행의 순서를 거꾸로 바꾸고 열은 그대로 유지하는 것을 의미합니다.

In [10]:
genre_sim_sorted_ind

array([[   0, 3494,  813, ..., 3038, 3037, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401],
       [   2, 1740, 1542, ..., 3000, 2999, 2401],
       ...,
       [4800, 3809, 1895, ..., 2229, 2230,    0],
       [4802, 1594, 1596, ..., 3204, 3205,    0],
       [4802, 4710, 4521, ..., 3140, 3141,    0]])

### **장르 콘텐츠 필터링을 이용한 영화 추천**

In [11]:
movies_df['title'].nunique()

4800

In [12]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
  # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
  title_movie = df[df['title'] == title_name]

  # title_name을 가진 DataFrame의 index 객체를 ndarray로 반환하고
  # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
  title_index = title_movie.index.values
  print(title_index)
  similar_indexes = sorted_ind[title_index, :(top_n)]

  # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임.
  #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
  print(similar_indexes)
  similar_indexes = similar_indexes.reshape(-1)

  return df.iloc[similar_indexes]

In [13]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average']]

[3337]
[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


장르와 키워드 기반으로만 분석을 진행했기에, 고객에게 추천하기 어려운 유명하지도 않고, 평점도 낮은 영화도 추천 순위 안에 들어가있는 것을 볼 수있음 => vote average column도 분석에 활용

### **vote average column을 사용해보기 위해 해당 칼럼을 확인**
- 1,2명의 관객이 만점 / 0점을 주어, 평점이 왜곡될 수 있기에 확인

In [14]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

# 거의 영화를 본 사람이 없는데, 한 두 사람이 보고 10점을 줘서 평균 점수가 높은 영화들이 존재함을 알 수 있다.

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


> 평점에 평가 횟수를 반영하는 새로운 평가 방식의 도입 => 평가 횟수에 대한 가중치

In [17]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:', round(C,3), 'm:',round(m,3))

C: 6.092 m: 370.2


In [20]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']

    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [21]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote', ascending = False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


**장르 유사성이 높은 영활르 top_n의 2배만큼의 후보군으로 선정한 뒤 weighted_vote 칼럼 값이 높은 순으로 정렬 후 top_n만큼 추출**

In [22]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
  title_movie = df[df['title'] == title_name]
  title_index = title_movie.index.values

  # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출
  similar_indexes = sorted_ind[title_index, :(top_n*2)]
  similar_indexes = similar_indexes.reshape(-1)

  # 기준 영화 index는 제외
  similar_indexes = similar_indexes[similar_indexes != title_index]

  # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출
  return df.iloc[similar_indexes].sort_values('weighted_vote', ascending = False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


=> 그럼 결국 추천 수가 더 많은 영화는 이전까지 받았던 추천에 의해 또 다른 사람들에게 추천이 되고 양극화가 심해진다는 것. 추천 알고리즘을 잘 활용하는 것, 처음의 리뷰를 좋게 받는 것 또한 매우 중요하다는 것을 알 수 있다.